# Player Re-ldentification in  Sports Footage  

#### Re-Identification in a Single Feed  

In [7]:
import numpy as np
import torch
import torchvision.transforms as transforms
from ultralytics import YOLO
import os
import json
from datetime import datetime
from collections import defaultdict, deque
import logging
from PIL import Image, ImageDraw, ImageFont
import imageio
from scipy.spatial.distance import cdist
import math

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class FeatureExtractor:
    def __init__(self):
        self.transform = transforms.Compose([
            transforms.Resize((128, 64)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def rgb_to_hsv(self, rgb_array):
        rgb_array = rgb_array.astype(np.float32) / 255.0
        r, g, b = rgb_array[:,:,0], rgb_array[:,:,1], rgb_array[:,:,2]
        max_val = np.maximum(np.maximum(r, g), b)
        min_val = np.minimum(np.minimum(r, g), b)
        diff = max_val - min_val
        h = np.zeros_like(max_val)
        mask = diff != 0
        r_mask = (max_val == r) & mask
        g_mask = (max_val == g) & mask
        b_mask = (max_val == b) & mask
        h[r_mask] = (60 * ((g[r_mask] - b[r_mask]) / diff[r_mask]) + 360) % 360
        h[g_mask] = (60 * ((b[g_mask] - r[g_mask]) / diff[g_mask]) + 120) % 360
        h[b_mask] = (60 * ((r[b_mask] - g[b_mask]) / diff[b_mask]) + 240) % 360
        s = np.zeros_like(max_val)
        s[max_val != 0] = diff[max_val != 0] / max_val[max_val != 0]
        v = max_val
        h = (h / 360 * 179).astype(np.uint8)
        s = (s * 255).astype(np.uint8)
        v = (v * 255).astype(np.uint8)
        return np.stack([h, s, v], axis=2)

    def calculate_histogram(self, channel, bins, range_vals):
        hist, _ = np.histogram(channel.flatten(), bins=bins, range=range_vals)
        if hist.sum() > 0:
            hist = hist.astype(np.float32) / hist.sum()
        return hist

    def extract_histogram_features(self, image_crop):
        if image_crop.size == 0:
            return np.zeros(671)
        hsv = self.rgb_to_hsv(image_crop)
        hist_h = self.calculate_histogram(hsv[:,:,0], 180, (0, 180))
        hist_s = self.calculate_histogram(hsv[:,:,1], 256, (0, 256))
        hist_v = self.calculate_histogram(hsv[:,:,2], 256, (0, 256))
        return np.concatenate([hist_h, hist_s, hist_v])

    def extract_lbp_features(self, image_crop):
        if image_crop.size == 0:
            return np.zeros(256)
        if len(image_crop.shape) == 3:
            gray = np.dot(image_crop[...,:3], [0.2989, 0.5870, 0.1140])
        else:
            gray = image_crop
        gray = gray.astype(np.uint8)
        rows, cols = gray.shape
        lbp = np.zeros_like(gray)
        for i in range(1, rows-1):
            for j in range(1, cols-1):
                center = gray[i, j]
                code = 0
                code |= (gray[i-1, j-1] > center) << 7
                code |= (gray[i-1, j] > center) << 6
                code |= (gray[i-1, j+1] > center) << 5
                code |= (gray[i, j+1] > center) << 4
                code |= (gray[i+1, j+1] > center) << 3
                code |= (gray[i+1, j] > center) << 2
                code |= (gray[i+1, j-1] > center) << 1
                code |= (gray[i, j-1] > center)
                lbp[i, j] = code
        return self.calculate_histogram(lbp, 256, (0, 256))

    def extract_features(self, image_crop):
        if image_crop.size == 0:
            return np.zeros(927)
        try:
            h, w = image_crop.shape[:2]
            if h < 32 or w < 16:
                pil_img = Image.fromarray(image_crop)
                pil_img = pil_img.resize((32, 64))
                image_crop = np.array(pil_img)
            hist_features = self.extract_histogram_features(image_crop)
            lbp_features = self.extract_lbp_features(image_crop)
            return np.concatenate([hist_features, lbp_features])
        except Exception as e:
            logger.warning(f"Feature extraction failed: {e}")
            return np.zeros(927)

class PlayerTracker:
    def __init__(self, max_disappeared=30, max_distance=100):
        self.next_id = 0
        self.players = {}
        self.disappeared = {}
        self.max_disappeared = max_disappeared
        self.max_distance = max_distance
        self.feature_extractor = FeatureExtractor()

    def register_player(self, centroid, bbox, features):
        self.players[self.next_id] = {
            'centroid': centroid,
            'bbox': bbox,
            'features': features,
            'history': deque([centroid], maxlen=10)
        }
        self.disappeared[self.next_id] = 0
        self.next_id += 1
        return self.next_id - 1

    def deregister_player(self, player_id):
        del self.players[player_id]
        del self.disappeared[player_id]

    def calculate_distance(self, point1, point2):
        return math.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

    def calculate_feature_similarity(self, f1, f2):
        if np.linalg.norm(f1) == 0 or np.linalg.norm(f2) == 0:
            return 0
        return np.dot(f1, f2) / (np.linalg.norm(f1) * np.linalg.norm(f2))

    def update(self, detections, frame):
        if len(detections) == 0:
            for pid in list(self.disappeared.keys()):
                self.disappeared[pid] += 1
                if self.disappeared[pid] > self.max_disappeared:
                    self.deregister_player(pid)
            return {}
        input_centroids = []
        input_features = []
        for (x1, y1, x2, y2, conf, cls) in detections:
            if cls == 0:
                centroid = (int((x1 + x2) / 2), int((y1 + y2) / 2))
                input_centroids.append(centroid)
                crop = frame[int(y1):int(y2), int(x1):int(x2)]
                features = self.feature_extractor.extract_features(crop)
                input_features.append(features)
        if len(self.players) == 0:
            for i, (centroid, features) in enumerate(zip(input_centroids, input_features)):
                bbox = detections[i][:4]
                self.register_player(centroid, bbox, features)
        else:
            player_ids = list(self.players.keys())
            D = np.zeros((len(input_centroids), len(player_ids)))
            for i, (c, f) in enumerate(zip(input_centroids, input_features)):
                for j, pid in enumerate(player_ids):
                    dist = self.calculate_distance(c, self.players[pid]['centroid'])
                    sim = self.calculate_feature_similarity(f, self.players[pid]['features'])
                    D[i][j] = dist * (1 - sim + 0.1)
            used_row_idxs, used_col_idxs = set(), set()
            for _ in range(min(len(input_centroids), len(player_ids))):
                (r, c) = np.unravel_index(np.argmin(D), D.shape)
                if D[r, c] > self.max_distance:
                    break
                if r in used_row_idxs or c in used_col_idxs:
                    D[r, c] = np.inf
                    continue
                pid = player_ids[c]
                self.players[pid]['centroid'] = input_centroids[r]
                self.players[pid]['bbox'] = detections[r][:4]
                self.players[pid]['features'] = input_features[r]
                self.players[pid]['history'].append(input_centroids[r])
                self.disappeared[pid] = 0
                used_row_idxs.add(r)
                used_col_idxs.add(c)
                D[r, c] = np.inf
            for r in set(range(len(input_centroids))) - used_row_idxs:
                self.register_player(input_centroids[r], detections[r][:4], input_features[r])
            for c in set(range(len(player_ids))) - used_col_idxs:
                pid = player_ids[c]
                self.disappeared[pid] += 1
                if self.disappeared[pid] > self.max_disappeared:
                    self.deregister_player(pid)
        return self.players

class PlayerReIdentification:
    def __init__(self, model_path="yolov8n.pt"):
        self.model_path = model_path
        self.model = None
        self.tracker = PlayerTracker()
        self.frame_count = 0
        self.results_log = []

    def load_model(self):
        try:
            if os.path.exists(self.model_path):
                self.model = YOLO(self.model_path)
                logger.info(f"Model loaded from {self.model_path}")
            else:
                logger.warning(f"Model file not found: {self.model_path}")
                self.model = YOLO("yolov8n.pt")
        except Exception as e:
            logger.error(f"Model loading failed: {e}")
            self.model = YOLO("yolov8n.pt")

    def detect_players(self, frame):
        if self.model is None:
            return []
        try:
            results = self.model(frame, verbose=False)
            detections = []
            for result in results:
                for box in result.boxes:
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                    conf = box.conf[0].cpu().numpy()
                    cls = int(box.cls[0].cpu().numpy())
                    if cls == 0 and conf > 0.5:
                        detections.append([x1, y1, x2, y2, conf, cls])
            return detections
        except Exception as e:
            logger.error(f"Detection failed: {e}")
            return []

    def draw_results(self, frame, players):
        result_frame = Image.fromarray(frame)
        draw = ImageDraw.Draw(result_frame)
        try:
            font = ImageFont.truetype("arial.ttf", 20)
        except:
            font = ImageFont.load_default()
        for pid, info in players.items():
            x1, y1, x2, y2 = info['bbox']
            c = info['centroid']
            draw.rectangle([int(x1), int(y1), int(x2), int(y2)], outline="green", width=2)
            draw.text((int(x1), int(y1) - 20), f"Player {pid}", fill="yellow", font=font)
            draw.ellipse([c[0]-4, c[1]-4, c[0]+4, c[1]+4], fill="red")
        return np.array(result_frame)

    def process_video(self, video_path, output_path=None, display=False):
        if not os.path.exists(video_path):
            logger.error(f"Video file not found: {video_path}")
            return
        self.load_model()
        reader = imageio.get_reader(video_path)
        fps = reader.get_meta_data().get('fps', 30)
        writer = imageio.get_writer(output_path, fps=fps) if output_path else None

        for frame_idx, frame in enumerate(reader):
            self.frame_count += 1
            detections = self.detect_players(frame)
            players = self.tracker.update(detections, frame)

            # ✅ Log player positions for each frame
            frame_data = {
                "frame": self.frame_count,
                "players": {
                    pid: {
                        "centroid": info["centroid"],
                        "bbox": [float(x) for x in info["bbox"]]
                    } for pid, info in players.items()
                }
            }
            self.results_log.append(frame_data)

            result_frame = self.draw_results(frame, players)
            if writer:
                writer.append_data(result_frame)

            logger.info(f"Processed Frame {self.frame_count}")

        if writer:
            writer.close()
        reader.close()
        self.save_results()

    def save_results(self, filename=None):
        if filename is None:
            filename = f"tracking_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
        with open(filename, 'w') as f:
            json.dump(self.results_log, f, indent=2)
        logger.info(f"Saved tracking results to {filename}")


# --- Run script ---
if __name__ == "__main__":
    video_path = r"C:\\Users\\KASIM\\Downloads\\15sec_input_720p (1).mp4"
    model_path = "C:\\Users\\KASIM\\Downloads\\best (1).pt"
    output_path ="C:\\Users\\KASIM\\Downloads\\Op\\output_video.mp4"
    display_results = False

    system = PlayerReIdentification(model_path=model_path)
    system.process_video(video_path, output_path, display=display_results)


INFO:__main__:Model loaded from C:\Users\KASIM\Downloads\best (1).pt
INFO:__main__:Processed Frame 1
INFO:__main__:Processed Frame 2
INFO:__main__:Processed Frame 3
INFO:__main__:Processed Frame 4
INFO:__main__:Processed Frame 5
INFO:__main__:Processed Frame 6
INFO:__main__:Processed Frame 7
INFO:__main__:Processed Frame 8
INFO:__main__:Processed Frame 9
INFO:__main__:Processed Frame 10
INFO:__main__:Processed Frame 11
INFO:__main__:Processed Frame 12
INFO:__main__:Processed Frame 13
INFO:__main__:Processed Frame 14
INFO:__main__:Processed Frame 15
INFO:__main__:Processed Frame 16
INFO:__main__:Processed Frame 17
INFO:__main__:Processed Frame 18
INFO:__main__:Processed Frame 19
INFO:__main__:Processed Frame 20
INFO:__main__:Processed Frame 21
INFO:__main__:Processed Frame 22
INFO:__main__:Processed Frame 23
INFO:__main__:Processed Frame 24
INFO:__main__:Processed Frame 25
INFO:__main__:Processed Frame 26
INFO:__main__:Processed Frame 27
INFO:__main__:Processed Frame 28
INFO:__main__:Pr